In [1]:
from dotenv import load_dotenv 

load_dotenv() 


True

In [2]:
from langgraph.graph import StateGraph, END 
from typing import TypedDict, Annotated 
import operator 
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage 
from langchain_openai import ChatOpenAI 
from langchain_community.tools.tavily_search import TavilySearchResults 


In [3]:
tool = TavilySearchResults(max_results=4) 
print(type(tool))
print(tool.name)

<class 'langchain_community.tools.tavily_search.tool.TavilySearchResults'>
tavily_search_results_json


In [4]:
class AgentState(TypedDict): 
    messages: Annotated[list[AnyMessage], operator.add]

In [10]:
class Agent: 
    def __init__(self, model, tools, system: str = ""):
        self.system = system 
        graph = StateGraph(AgentState) 
        graph.add_node("llm", self.call_openai)  
        graph.add_node("action", self.take_action)
        graph.add_conditional_edges(
            "llm",
            self.exists_action,
            { True: "action", False: END }
        )
        graph.add_edge("action", "llm")
        graph.set_entry_point("llm")
        self.graph = graph.compile() 
        self.tools = {t.name: t for t in tools}
        self.model = model.bind_tools(tools)
    
    def exists_action(self, state: AgentState): 
        result = state['messages'][-1] 
        return len(result.tool_calls) > 0 

    def take_action(self, state: AgentState): 
        tool_calls = state['messages'][-1].tool_calls 
        results = [] 
        for t in tool_calls: 
            print(f"Calling: {t}") 
            if not t['name'] in self.tools: 
                print("\n... Bad tool Name ...") 
                result = "bad tool name, retry"
            else: 
                result = self.tools[t['name']].invoke(t['args']) 
            results.append(ToolMessage(tool_call_id=t['id'], name=t['name'], content=str(result)))
        return {'messages': results}

    def call_openai(self, state: AgentState): 
        messages = state['messages']
        if self.system: 
            messages = [SystemMessage(content=self.system)] + messages 
        message = self.model.invoke(messages)
        return {'messages': [message]}

In [11]:
prompt = """You are a smart research assistant. Use the search engine to look up information. \
You are allowed to make multiple calls (either together or in sequence). \
Only look up information when you are sure of what you want. \
If you need to look up some information before asking a follow up question, you are allowed to do that!
"""
import os
model = ChatOpenAI(model="gpt-4o", api_key=os.getenv('OPENAI_API_KEY'))  #reduce inference cost
abot = Agent(model, [tool], system=prompt)

In [12]:
# from IPython.display import Image

# Image(abot.graph.get_graph().draw_png())

In [13]:
messages = [HumanMessage(content="What is the weather in sf?")]
result = abot.graph.invoke({"messages": messages})

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'San Francisco weather current'}, 'id': 'call_dqaIiCOvcDPnxDYOjqH0IFiY', 'type': 'tool_call'}


In [16]:
result['messages'][-1].content

'The current weather in San Francisco is clear with a temperature of 10.1°C (50.2°F). The wind is blowing from the north-northeast at 5.4 mph (8.6 kph), and the humidity level is 50%. The visibility is good, with a distance of 16 km (9 miles).'

In [17]:
messages = [HumanMessage(content="What is the weather in SF and LA?")]
result = abot.graph.invoke({"messages": messages})

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'current weather in San Francisco'}, 'id': 'call_XJo7Be6CBgzsqVEL9oRTjFIh', 'type': 'tool_call'}
Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Los Angeles'}, 'id': 'call_OG9L4omsY6WNEa3WmcKFjsxh', 'type': 'tool_call'}


In [18]:
result['messages'][-1].content

'The current weather in San Francisco is clear with a temperature of 10.1°C (50.2°F). The wind is blowing from the north-northeast at 5.4 mph (8.6 kph), and the humidity is at 50%.\n\nIn Los Angeles, the weather is also clear, with a temperature of 13.3°C (55.9°F). The wind is coming from the north-northeast at 6.9 mph (11.2 kph), and the humidity is at 20%.'

In [19]:
# Note, the query was modified to produce more consistent results. 
# Results may vary per run and over time as search information and models change.

query = "Who won the super bowl in 2024? In what state is the winning team headquarters located? \
What is the GDP of that state? Answer each question." 
messages = [HumanMessage(content=query)]

model = ChatOpenAI(model="gpt-4o")  # requires more advanced model
abot = Agent(model, [tool], system=prompt)
result = abot.graph.invoke({"messages": messages})

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'Super Bowl 2024 winner'}, 'id': 'call_p7CUNJaQ2zqXNp8oy0jBntkF', 'type': 'tool_call'}
Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'Kansas City Chiefs headquarters location'}, 'id': 'call_ghZ8tyEfqArdF8T89rfiVBng', 'type': 'tool_call'}
Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'Missouri GDP 2024'}, 'id': 'call_VoQKyjIrv46CaWNDgKWRxiPs', 'type': 'tool_call'}


In [20]:
print(result['messages'][-1].content)

1. **Who won the Super Bowl in 2024?**  
   The Kansas City Chiefs won the Super Bowl in 2024, defeating the San Francisco 49ers 25-22 in overtime.

2. **In what state is the winning team's headquarters located?**  
   The headquarters of the Kansas City Chiefs is located in Missouri.

3. **What is the GDP of that state?**  
   As of the third quarter of 2024, Missouri's GDP was approximately $454.58 billion.
